In [0]:
import numpy as np
import spacy
import re

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
worksheet = gc.open('Copy of DatafinitiElectronicsProductsPricingData').sheet1

In [0]:
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
electronics = pd.DataFrame.from_records(rows)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
header = electronics.iloc[0]
electronics = electronics[1:]
electronics.columns = header
electronics.head()

,id,prices.amountMax,prices.amountMin,prices.availability,prices.condition,prices.currency,prices.dateSeen,prices.isSale,prices.merchant,prices.shipping,prices.sourceURLs,asins,brand,categories,dateAdded,dateUpdated,ean,imageURLs,keys,manufacturer,manufacturerNumber,name,primaryCategories,sourceURLs,upc,weight,,,,,
1,AVphzgbJLJeJML43fA0o,104.99,104.99,Yes,New,USD,"2017-03-30T06:00:00Z,2017-03-10T22:00:00Z,2017...",FALSE,Bestbuy.com,,http://www.bestbuy.com/site/sanus-tv-wall-moun...,B00C78VIUE,Sanus,"Audio & Video Accessories,TV Mounts,TV Accesso...",2015-04-13T12:00:51Z,2018-05-12T18:59:48Z,,https://images-na.ssl-images-amazon.com/images...,sanusvlf410b110inchsuperslimfullmotionmountfor...,,VLF410B1,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,Electronics,https://www.amazon.com/Sanus-VLF410B1-10-Inch-...,7.94E+11,32.8 pounds,,,,,
2,AVpgMuGwLJeJML43KY_c,69,64.99,In Stock,New,USD,2017-12-14T06:00:00Z,TRUE,Walmart.com,Expedited,https://www.walmart.com/ip/BOYTONE-BT210FB-BLA...,"B018K251JE,B00VILQKQ8",Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,,,,,
3,AVpgMuGwLJeJML43KY_c,69,69,In Stock,New,USD,2017-09-08T05:00:00Z,FALSE,Walmart.com,Expedited,https://www.walmart.com/ip/Boytone-BT-210F-Blu...,"B018K251JE,B00VILQKQ8",Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,,,,,
4,AVpgMuGwLJeJML43KY_c,69.99,69.99,Yes,New,USD,2017-10-10T05:00:00Z,FALSE,Bestbuy.com,,https://www.bestbuy.com/site/boytone-2500w-2-1...,"B018K251JE,B00VILQKQ8",Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,,,,,
5,AVpgMuGwLJeJML43KY_c,66.99,66.99,Yes,New,USD,2017-08-28T07:00:00Z,FALSE,Bestbuy.com,,https://www.bestbuy.com/site/boytone-2500w-2-1...,"B018K251JE,B00VILQKQ8",Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,,,,,


In [0]:
electronics.head()

,id,prices.amountMax,prices.amountMin,prices.availability,prices.condition,prices.currency,prices.dateSeen,prices.isSale,prices.merchant,prices.shipping,prices.sourceURLs,asins,brand,categories,dateAdded,dateUpdated,ean,imageURLs,keys,manufacturer,manufacturerNumber,name,primaryCategories,sourceURLs,upc,weight,,,,,
1,AVphzgbJLJeJML43fA0o,104.99,104.99,Yes,New,USD,"2017-03-30T06:00:00Z,2017-03-10T22:00:00Z,2017...",FALSE,Bestbuy.com,,http://www.bestbuy.com/site/sanus-tv-wall-moun...,B00C78VIUE,Sanus,"Audio & Video Accessories,TV Mounts,TV Accesso...",2015-04-13T12:00:51Z,2018-05-12T18:59:48Z,,https://images-na.ssl-images-amazon.com/images...,sanusvlf410b110inchsuperslimfullmotionmountfor...,,VLF410B1,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,Electronics,https://www.amazon.com/Sanus-VLF410B1-10-Inch-...,7.94E+11,32.8 pounds,,,,,
2,AVpgMuGwLJeJML43KY_c,69,64.99,In Stock,New,USD,2017-12-14T06:00:00Z,TRUE,Walmart.com,Expedited,https://www.walmart.com/ip/BOYTONE-BT210FB-BLA...,"B018K251JE,B00VILQKQ8",Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,,,,,
3,AVpgMuGwLJeJML43KY_c,69,69,In Stock,New,USD,2017-09-08T05:00:00Z,FALSE,Walmart.com,Expedited,https://www.walmart.com/ip/Boytone-BT-210F-Blu...,"B018K251JE,B00VILQKQ8",Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,,,,,
4,AVpgMuGwLJeJML43KY_c,69.99,69.99,Yes,New,USD,2017-10-10T05:00:00Z,FALSE,Bestbuy.com,,https://www.bestbuy.com/site/boytone-2500w-2-1...,"B018K251JE,B00VILQKQ8",Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,,,,,
5,AVpgMuGwLJeJML43KY_c,66.99,66.99,Yes,New,USD,2017-08-28T07:00:00Z,FALSE,Bestbuy.com,,https://www.bestbuy.com/site/boytone-2500w-2-1...,"B018K251JE,B00VILQKQ8",Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ...",2015-05-18T14:14:56Z,2018-06-13T19:39:02Z,,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,,,,,


In [0]:
electronics['sub_category'] = electronics.categories.str.extract('^(.+?),')

In [0]:
electronics.sub_category.isnull().sum()

0

In [0]:
#delete non-essential columns
#electronics2 = electronics.iloc[:, [0,1,12,21,22,25,26,27]]




In [0]:
#electronics2 = electronics[['id', 'prices.amountMax', 'brand', 'name', 'primaryCategories',
            'weight', 'sub_category']]

In [0]:
electronics2 = electronics[['id', 'prices.amountMax', 'brand', 'name', 'primaryCategories',
            'sub_category']]

In [0]:
electronics2.head()

,id,price,brand,description,categories,sub_category
1,AVphzgbJLJeJML43fA0o,104.99,Sanus,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,Electronics,Audio & Video Accessories
2,AVpgMuGwLJeJML43KY_c,69,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,Stereos
3,AVpgMuGwLJeJML43KY_c,69,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,Stereos
4,AVpgMuGwLJeJML43KY_c,69.99,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,Stereos
5,AVpgMuGwLJeJML43KY_c,66.99,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,Stereos


In [0]:
#rename columns
electronics2.columns = ['id', 'price', 'brand', 'description', 'categories',
            'sub_category']

In [0]:
electronics2[['Weight','Unit']] = electronics2.weight.str.split(n=1, expand=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [0]:
electronics2['Unit'] = electronics2.Unit.str.extract(r'(^\w+)')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
for col in electronics2:
    print(electronics2['Unit'].unique())

['pounds' 'lb' 'ounces' 'oz' 'lbs' 'g' 'Kg' 'kg' 'Pounds' nan]
['pounds' 'lb' 'ounces' 'oz' 'lbs' 'g' 'Kg' 'kg' 'Pounds' nan]
['pounds' 'lb' 'ounces' 'oz' 'lbs' 'g' 'Kg' 'kg' 'Pounds' nan]
['pounds' 'lb' 'ounces' 'oz' 'lbs' 'g' 'Kg' 'kg' 'Pounds' nan]
['pounds' 'lb' 'ounces' 'oz' 'lbs' 'g' 'Kg' 'kg' 'Pounds' nan]
['pounds' 'lb' 'ounces' 'oz' 'lbs' 'g' 'Kg' 'kg' 'Pounds' nan]
['pounds' 'lb' 'ounces' 'oz' 'lbs' 'g' 'Kg' 'kg' 'Pounds' nan]
['pounds' 'lb' 'ounces' 'oz' 'lbs' 'g' 'Kg' 'kg' 'Pounds' nan]
['pounds' 'lb' 'ounces' 'oz' 'lbs' 'g' 'Kg' 'kg' 'Pounds' nan]


In [0]:
electronics2.Unit.value_counts()

pounds    3111
lb        1535
ounces    1377
oz         819
lbs        312
g           38
Kg          32
Pounds       7
kg           6
Name: Unit, dtype: int64

In [0]:
electronics2.Weight.astype(float)

1       32.8
2       14.0
3       14.0
4       14.0
5       14.0
        ... 
7245     1.8
7246     1.8
7247     1.8
7248     1.8
7249     1.8
Name: Weight, Length: 7249, dtype: float64

In [0]:
def convert(weight):
    if electronics2["unit"] == "pounds" or electronics2["unit"] == "lbs" or electronics2["unit"] == "lb":
        return electronics2["weight"] == electronics2["weight"] * 454
    elif electronics2["unit"] == "ounces" or electronics2["unit"] == "oz":
        return electronics2["unit"] == electronics2["unit"] * 28.35

In [0]:
worksheet = gc.open('amazon_co-ecommerce_sample').sheet1
rows = worksheet.get_all_values()
toys = pd.DataFrame.from_records(rows)

In [0]:
toys.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
1,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
2,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,,2,1,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
3,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
4,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,,1,2,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,,,I love it // 5.0 // 22 July 2013 // By\n \n...,


In [0]:
header = toys.iloc[0]
toys = toys[1:]
toys.columns = header
toys.head()

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
1,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
2,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,,2,1,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
3,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
4,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,,1,2,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,,,I love it // 5.0 // 22 July 2013 // By\n \n...,
5,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,,3,2,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,,Birthday present // 5.0 // 14 April 2014 // By...,


In [0]:
#toys['price'] = toys['price'].replace('£',"",regex= True)


In [0]:
toys.price.isnull().values.any()

False

In [0]:
def number_split(i):
    """" Split the numbers in the column
         with a hyphen and take the average of 
        two numbers and return the average"""
    average = 0
    splt = i.split('–')
    if len(splt)==2:
        first = float(splt[0])
        second = float(splt[1])
        average = (first+second)/2
    else:
        average = float(splt[0])


In [0]:
pd.options.display.max_rows = 15000
pd.options.display.max_columns = 15000
toys.price

1                      £3.42
2                     £16.99
3                      £9.99
4                     £39.99
5                     £32.19
6                      £6.99
7                     £24.99
8                     £69.93
9                    £235.58
10                          
11                    £27.49
12                   £273.60
13                     £9.60
14                   £119.50
15                          
16                          
17                          
18                          
19                    £17.08
20                    £96.05
21                    £27.55
22                    £24.50
23                   £149.92
24                    £12.87
25                    £49.95
26                          
27                    £14.44
28                          
29                          
30                     £5.95
31                   £139.95
32                    £19.99
33                    £25.50
34                          
35            

In [0]:
toys.head(2)

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
1,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,4.0,5 new,15,1,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
2,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,21.0,,2,1,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."


In [0]:
def convert_to_usd(price):
    try:
        price = round(float(price[1:]) * 1.23)
        return price
    except:
        return np.nan

In [0]:
toys['price'] = toys['price'].apply(convert_to_usd)

In [0]:
toys['price'] = toys['price'].apply(number_split)

In [0]:
toys = toys.dropna(subset=['price'])

In [0]:
toys.manufacturer.value_counts()

Oxford Diecast                  152
LEGO                            138
Disney                          136
Playmobil                       120
The Puppet Company              102
                               ... 
The Balloon Printing Company      1
Adrenalyn                         1
North Star Games                  1
hongwell                          1
Giotto                            1
Name: manufacturer, Length: 2363, dtype: int64

In [0]:
toys2 = toys.iloc[:, [0,1,2,3,8,11,12]]
toys2.head()

,uniq_id,product_name,manufacturer,price,amazon_category_and_sub_category,product_information,product_description
1,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,4.0,Hobbies > Model Trains & Railway Sets > Rail V...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...
2,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,21.0,Hobbies > Model Trains & Railway Sets > Rail V...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...
3,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,12.0,Hobbies > Model Trains & Railway Sets > Rail V...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...
4,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,49.0,Hobbies > Model Trains & Railway Sets > Rail V...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...
5,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,40.0,Hobbies > Model Trains & Railway Sets > Rail V...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...


In [0]:
toys2['sub_category'] = toys2.amazon_category_and_sub_category.str.extract('\>(.*?)\>')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
toys2.loc[toys2['sub_category'].isnull(), 'sub_category'] = 'unk'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [0]:
toys2.isnull().sum()

0
uniq_id                             0
product_name                        0
manufacturer                        0
price                               0
amazon_category_and_sub_category    0
product_information                 0
product_description                 0
sub_category                        0
category                            0
dtype: int64

In [0]:
electronics2.isnull().sum()

id              0
price           0
brand           0
description     0
categories      0
sub_category    0
dtype: int64

In [0]:
toys2['category'] = 'Toys'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
toys2["sub_category"].isin(["unk"]).sum()

3832

In [0]:
toys2

,id,price,brand,description,categories,sub_category
1,eac7efa5dbd3d667f26eb3d3ab504464,4.0,Hornby,Product Description Hornby 2014 Catalogue Box ...,Toys,Model Trains & Railway Sets
2,b17540ef7e86e461d37f3ae58b7b72ac,21.0,FunkyBuys,Size Name:Large FunkyBuys® Large Christmas Hol...,Toys,Model Trains & Railway Sets
3,348f344247b0c1a935b1223072ef9d8a,12.0,ccf,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Toys,Model Trains & Railway Sets
4,e12b92dbb8eaee78b22965d2a9bbbd9f,49.0,Hornby,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Toys,Model Trains & Railway Sets
5,e33a9adeed5f36840ccc227db4682a36,40.0,Hornby,Product Description Hornby RailRoad 0-4-0 Gild...,Toys,Model Trains & Railway Sets
...,...,...,...,...,...,...
9996,44d6967f083825a5de36ad4865a65bcd,28.0,Mattel,DC 66 Batman Classic TV Series 6 Inch Riddler ...,Toys,Collectible Figures & Memorabilia
9997,08f0747b6fc6687215ffb994c3a6fb32,49.0,Star Wars,,Toys,Star Wars
9998,bf6cc073f8f24e6e338190fa16f6ee9d,54.0,Olde Scotland Yard Ltd.,*Includes 1x Badge with holder and chain *High...,Toys,Novelty
9999,cd783d0b8b44e631b9788b203eaaefae,61.0,DC Comics,Designed by Ed BenesIt's here - the third seri...,Toys,Collectible Figures & Memorabilia


In [0]:
#dropped columns
toys2 = toys2[['uniq_id', 'price', 'manufacturer', 'product_description', 'category',
            'sub_category']]

In [0]:
#rename columns
toys2.columns = ['id', 'price', 'brand', 'description', 'categories',
            'sub_category']

In [0]:
electronics2.shape

(7249, 6)

In [0]:
final_df = toys2.append(electronics2, ignore_index = True) 

In [0]:
final_df.tail()

,id,price,brand,description,categories,sub_category
15790,AVpfUsrxLJeJML437Ezd,79.99,Lowepro,Lowepro - Slingshot Edge 250 AW Camera Backpac...,Electronics,Camera & Photo Accessories
15791,AVpfUsrxLJeJML437Ezd,68.75,Lowepro,Lowepro - Slingshot Edge 250 AW Camera Backpac...,Electronics,Camera & Photo Accessories
15792,AVpfUsrxLJeJML437Ezd,70.99,Lowepro,Lowepro - Slingshot Edge 250 AW Camera Backpac...,Electronics,Camera & Photo Accessories
15793,AVpfUsrxLJeJML437Ezd,71.99,Lowepro,Lowepro - Slingshot Edge 250 AW Camera Backpac...,Electronics,Camera & Photo Accessories
15794,AVpfUsrxLJeJML437Ezd,67.77,Lowepro,Lowepro - Slingshot Edge 250 AW Camera Backpac...,Electronics,Camera & Photo Accessories


In [0]:
#final_df['description'].fillna('unk')
final_df = final_df.fillna('unk')
#final_df['brand'].fillna('unk')
#final_df['categories'].fillna('unk')

In [0]:
final_df.loc[final_df['brand'].isnull(), 'brand'] = 'unk'
final_df.loc[final_df['description'].isnull(), 'description'] = 'unk'
final_df.loc[final_df['categories'].isnull(), 'categories'] = 'unk'

In [0]:
final_df.dtypes

id               object
price           float64
brand            object
description      object
categories       object
sub_category     object
dtype: object

In [0]:
final_df['price'] = final_df['price'].astype(float)

In [0]:
unknown = final_df["description"].isin(["unk"]).sum()
unknown

0

In [0]:
final_df.isnull().sum()

id              0
price           0
brand           0
description     0
categories      0
sub_category    0
dtype: int64

In [0]:
final_df.to_csv('data_nlp.csv', index=False)
!cp data_nlp.csv "/content/drive/My Drive/"

In [0]:
/content/drive/My Drive/

In [0]:
#join together brand des cate sub
#train, test split
#count vectorizer
#model

In [0]:
final_df.head()

,id,price,brand,description,categories,sub_category
0,eac7efa5dbd3d667f26eb3d3ab504464,4.0,Hornby,Product Description Hornby 2014 Catalogue Box ...,Toys,Model Trains & Railway Sets
1,b17540ef7e86e461d37f3ae58b7b72ac,21.0,FunkyBuys,Size Name:Large FunkyBuys® Large Christmas Hol...,Toys,Model Trains & Railway Sets
2,348f344247b0c1a935b1223072ef9d8a,12.0,ccf,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Toys,Model Trains & Railway Sets
3,e12b92dbb8eaee78b22965d2a9bbbd9f,49.0,Hornby,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Toys,Model Trains & Railway Sets
4,e33a9adeed5f36840ccc227db4682a36,40.0,Hornby,Product Description Hornby RailRoad 0-4-0 Gild...,Toys,Model Trains & Railway Sets


In [0]:
from tqdm import tqdm